In [22]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.datasets import imdb

In [23]:
max_features = 5000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [24]:
print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [25]:
print('Pad sequences (samples x time)') 
X_train = sequence.pad_sequences(X_train, maxlen=maxlen) 
X_test = sequence.pad_sequences(X_test, maxlen=maxlen) 

Pad sequences (samples x time)


In [26]:
set(y_test)

{0, 1}

In [27]:
if False:
    from itertools import chain
    max_features = len(set(chain.from_iterable(X_train)) and set(chain.from_iterable(X_test)))

In [28]:
from keras.layers import Convolution1D , GlobalMaxPooling1D
from keras.layers.convolutional import Conv1D
num_filters = 150
kernel_size = 3
filter_length = 3

むしろ
http://qiita.com/HirofumiYashima/items/2aee883b736ee600899b
あるいは
http://qiita.com/HirofumiYashima/items/a01902cb7d3e74b56bcb
を参考にCNNを構成しよう。後者がstraightforwardか？？

In [29]:
model = Sequential()
model.add(Embedding(input_dim = max_features, output_dim = 50, dropout=0.2))

Convolution1Dはobsoleteのようだがとりあえず、これで実装する。（している例が多い）
- nb_filter : アウトプット次元（＝フィルターの数）
- filter_length :ウィンドウのサイズ(trinomial相当なら3）

In [30]:
model.add(Convolution1D(nb_filter = num_filters , filter_length = filter_length , activation = "relu"))
#model.add(Conv1D(filter = num_filters , kernel_size = kernel_size))

In [31]:
Convolution1D?

In [32]:
# we use max pooling: 
model.add(GlobalMaxPooling1D()) 

hidden_dims = 250 
# We add a vanilla hidden layer: 
model.add(Dense(hidden_dims)) 
model.add(Dropout(0.2)) 
model.add(Activation('relu')) 
# We project onto a single unit output layer, and squash it with a sigmoid: 
model.add(Dense(1)) 
model.add(Activation('sigmoid')) 

In [33]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, None, 50)      250000      embedding_input_3[0][0]          
____________________________________________________________________________________________________
convolution1d_3 (Convolution1D)  (None, None, 150)     22650       embedding_3[0][0]                
____________________________________________________________________________________________________
globalmaxpooling1d_3 (GlobalMaxPo(None, 150)           0           convolution1d_3[0][0]            
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 250)           37750       globalmaxpooling1d_3[0][0]       
___________________________________________________________________________________________

In [34]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']) 

In [35]:
nb_epoch = 1
model.fit(X_train, y_train, 
          batch_size=batch_size, 
          nb_epoch=nb_epoch, 
          validation_data=(X_test, y_test) , verbose = 2) 

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
1183s - loss: 0.4902 - acc: 0.7499 - val_loss: 0.3760 - val_acc: 0.8323


以下、型チェック用：

In [13]:
model.fit?

In [ ]:
score = model.evaluate(X_test, y_test, verbose=2)
print('Test score:', score[0])
print('Test accuracy:', score[1])